# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import pandas as pd
import numpy as np

import patsy
import statsmodels.api as sm

In [2]:
df = pd.read_csv("C:\\Users\\alcid\\GitHub\\Portfolio\\EBAC\\Dados\\previsao_de_renda.csv")

In [3]:
# Verificar se há dados faltantes e os tipos de dados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   index                  15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12466 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  mau                    15000 non-null  bool   
 15  re

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.

2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.

3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [4]:
# Verificar se há dados duplicados
df.duplicated().sum()

0

In [5]:
df = df.dropna() # Excluir dados faltantes
df = df.drop('Unnamed: 0', axis=1) # Eliminar coluna inútil 
id = df.pop('index') # Remover o índice.
df.info() # Informações do dataset alterado.
pd.options.display.float_format = '{:.2f}'.format

<class 'pandas.core.frame.DataFrame'>
Index: 12466 entries, 0 to 14999
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   data_ref               12466 non-null  object 
 1   sexo                   12466 non-null  object 
 2   posse_de_veiculo       12466 non-null  bool   
 3   posse_de_imovel        12466 non-null  bool   
 4   qtd_filhos             12466 non-null  int64  
 5   tipo_renda             12466 non-null  object 
 6   educacao               12466 non-null  object 
 7   estado_civil           12466 non-null  object 
 8   tipo_residencia        12466 non-null  object 
 9   idade                  12466 non-null  int64  
 10  tempo_emprego          12466 non-null  float64
 11  qt_pessoas_residencia  12466 non-null  float64
 12  mau                    12466 non-null  bool   
 13  renda                  12466 non-null  float64
dtypes: bool(3), float64(3), int64(2), object(6)
memory usage: 1

In [6]:
df.head()

,data_ref,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,mau,renda
0,2015-01-01,F,False,True,1,Assalariado,Secundário,Casado,Casa,36,3.58,3.00,False,3369.24
1,2015-01-01,M,True,True,0,Empresário,Secundário,Casado,Casa,42,0.86,2.00,False,6096.14
2,2015-01-01,M,True,True,2,Assalariado,Superior completo,Casado,Casa,31,8.07,4.00,False,5658.98
3,2015-01-01,F,True,False,0,Empresário,Secundário,Casado,Casa,50,1.21,2.00,False,7246.69
4,2015-01-01,M,False,False,0,Assalariado,Secundário,Casado,Casa,52,13.87,2.00,False,4017.37


In [7]:
df.describe().T


,count,mean,std,min,25%,50%,75%,max
qtd_filhos,12466.00,0.52,0.80,0.00,0.00,0.00,1.00,14.00
idade,12466.00,40.70,9.36,22.00,33.00,40.00,48.00,67.00
tempo_emprego,12466.00,7.75,6.78,0.19,3.02,6.02,10.18,42.91
qt_pessoas_residencia,12466.00,2.31,0.96,1.00,2.00,2.00,3.00,15.00
renda,12466.00,4850.05,4873.18,159.90,2026.39,3422.44,5916.97,89918.04


In [8]:
df[df.describe().T.index].nunique()

qtd_filhos                  8
idade                      46
tempo_emprego            2571
qt_pessoas_residencia       9
renda                    8114
dtype: int64

In [9]:
# Verificar frequência dos numéricos com poucas classes.
for atributo in ['qtd_filhos','qt_pessoas_residencia']:
    print(df[atributo].value_counts().index[0])

0
2.0


In [10]:
df.describe(exclude=np.number).T

,count,unique,top,freq
data_ref,12466,15,2015-05-01,848
sexo,12466,2,F,7991
posse_de_veiculo,12466,2,False,7215
posse_de_imovel,12466,2,True,8127
tipo_renda,12466,5,Assalariado,7826
educacao,12466,5,Secundário,7161
estado_civil,12466,5,Casado,8896
tipo_residencia,12466,6,Casa,11128
mau,12466,2,False,12165


In [11]:
# Verificar frequência dos categoricos 
for atributo in df.select_dtypes(exclude=np.number):
    print(atributo, df[atributo].value_counts().index[0], df[atributo].unique())

data_ref 2015-05-01 ['2015-01-01' '2015-02-01' '2015-03-01' '2015-04-01' '2015-05-01'
 '2015-06-01' '2015-07-01' '2015-08-01' '2015-09-01' '2015-10-01'
 '2015-11-01' '2015-12-01' '2016-01-01' '2016-02-01' '2016-03-01']
sexo F ['F' 'M']
posse_de_veiculo False [False  True]
posse_de_imovel True [ True False]
tipo_renda Assalariado ['Assalariado' 'Empresário' 'Servidor público' 'Bolsista' 'Pensionista']
educacao Secundário ['Secundário' 'Superior completo' 'Superior incompleto' 'Primário'
 'Pós graduação']
estado_civil Casado ['Casado' 'União' 'Solteiro' 'Separado' 'Viúvo']
tipo_residencia Casa ['Casa' 'Com os pais' 'Aluguel' 'Governamental' 'Estúdio' 'Comunitário']
mau False [False  True]


In [12]:
# Separando logo os atributos numéricos
numericos = 'idade + tempo_emprego + C(qtd_filhos, Treatment(0)) + C(qt_pessoas_residencia, Treatment(2))'
categoricos = 'C(data_ref, Treatment(0)) + C(sexo, Treatment(0)) +  C(posse_de_veiculo, Treatment(0))'
categoricos += ' + C(posse_de_imovel, Treatment(1)) + C(tipo_renda, Treatment(0)) + C(educacao, Treatment(2))'
categoricos += '+ C(estado_civil, Treatment(0)) + C(tipo_residencia, Treatment(1)) + C(mau, Treatment(0))'
atributos = f'np.log(renda) ~ {numericos} + {categoricos}'
y, X = patsy.dmatrices(atributos, df)
# Treinando o modeo
model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.244
Model:                            OLS   Adj. R-squared:                  0.241
Method:                 Least Squares   F-statistic:                     81.64
Date:                Fri, 27 Oct 2023   Prob (F-statistic):               0.00
Time:                        16:44:13   Log-Likelihood:                -13188.
No. Observations:               12466   AIC:                         2.648e+04
Df Residuals:                   12416   BIC:                         2.685e+04
Df Model:                          49                                         
Covariance Type:            nonrobust                                         
=====================================================================================================================
                                                        coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------
Intercept                                             7.3234      0.044    165.578      0.000       7.237       7.410
C(qtd_filhos, Treatment(0))[T.1]                     -0.0388      0.131     -0.297      0.766      -0.295       0.218
C(qtd_filhos, Treatment(0))[T.2]                      0.0995      0.268      0.372      0.710      -0.425       0.624
C(qtd_filhos, Treatment(0))[T.3]                     -0.4693      0.472     -0.995      0.320      -1.394       0.455
C(qtd_filhos, Treatment(0))[T.4]                     -0.1715      0.415     -0.414      0.679      -0.984       0.641
C(qtd_filhos, Treatment(0))[T.5]                     -0.0295      0.399     -0.074      0.941      -0.811       0.752
C(qtd_filhos, Treatment(0))[T.7]                     -0.4779      0.247     -1.933      0.053      -0.962       0.007
C(qtd_filhos, Treatment(0))[T.14]                     0.0551      0.187      0.295      0.768      -0.311       0.421
C(qt_pessoas_residencia, Treatment(2))[T.1.0]        -0.0095      0.129     -0.074      0.941      -0.263       0.244
C(qt_pessoas_residencia, Treatment(2))[T.3.0]         0.0411      0.131      0.313      0.754      -0.216       0.298
C(qt_pessoas_residencia, Treatment(2))[T.4.0]        -0.0415      0.268     -0.155      0.877      -0.567       0.484
C(qt_pessoas_residencia, Treatment(2))[T.5.0]         0.4922      0.475      1.036      0.300      -0.439       1.423
C(qt_pessoas_residencia, Treatment(2))[T.6.0]         0.3847      0.399      0.965      0.335      -0.397       1.166
C(qt_pessoas_residencia, Treatment(2))[T.7.0]        -0.5857      0.638     -0.918      0.359      -1.837       0.665
C(qt_pessoas_residencia, Treatment(2))[T.9.0]        -0.4779      0.247     -1.933      0.053      -0.962       0.007
C(qt_pessoas_residencia, Treatment(2))[T.15.0]        0.0551      0.187      0.295      0.768      -0.311       0.421
C(data_ref, Treatment(0))[T.2015-02-01]               0.0438      0.034      1.279      0.201      -0.023       0.111
C(data_ref, Treatment(0))[T.2015-03-01]               0.0511      0.034      1.493      0.135      -0.016       0.118
C(data_ref, Treatment(0))[T.2015-04-01]               0.0456      0.034      1.329      0.184      -0.022       0.113
C(data_ref, Treatment(0))[T.2015-05-01]               0.0129      0.034      0.380      0.704      -0.054       0.080
C(data_ref, Treatment(0))[T.2015-06-01]               0.0324      0.034      0.942      0.346      -0.035       0.100
C(data_ref, Treatment(0))[T.2015-07-01]               0.0254      0.034      0.737      0.461      -0.042       0.093
C(data_ref, Treatment(0))[T.2015-08-01]              -0.0138      0.034     -0.402      0.688      -0.081       0.054
C(data_ref, Tre

In [13]:
# Para facilitar o estudo, irei extrair a tabela com as informações estatísticas de summary.
resultado = pd.DataFrame(model.summary().tables[1],)
resultado.iloc[0,0] = 'indice'
resultado.columns = list(map(lambda x: str(x), resultado.iloc[0,:]))
resultado = resultado.drop(resultado.index[0])

for i in resultado:
    resultado[i] = resultado[i].astype(str)
    
resultado = resultado.set_index('indice') 
for i in resultado:
    resultado[i] = resultado[i].astype(float)
    
resultado.sort_values(by='P>|t|',ascending=False)
resultado.loc[resultado['P>|t|'] > 0.1]

,coef,std err,t,P>|t|,[0.025,0.975]
indice,,,,,,
"C(qtd_filhos, Treatment(0))[T.1]",-0.04,0.13,-0.30,0.77,-0.29,0.22
"C(qtd_filhos, Treatment(0))[T.2]",0.10,0.27,0.37,0.71,-0.42,0.62
"C(qtd_filhos, Treatment(0))[T.3]",-0.47,0.47,-0.99,0.32,-1.39,0.46
"C(qtd_filhos, Treatment(0))[T.4]",-0.17,0.41,-0.41,0.68,-0.98,0.64
"C(qtd_filhos, Treatment(0))[T.5]",-0.03,0.40,-0.07,0.94,-0.81,0.75
"C(qtd_filhos, Treatment(0))[T.14]",0.06,0.19,0.29,0.77,-0.31,0.42
"C(qt_pessoas_residencia, Treatment(2))[T.1.0]",-0.01,0.13,-0.07,0.94,-0.26,0.24
"C(qt_pessoas_residencia, Treatment(2))[T.3.0]",0.04,0.13,0.31,0.75,-0.22,0.30
"C(qt_pessoas_residencia, Treatment(2))[T.4.0]",-0.04,0.27,-0.15,0.88,-0.57,0.48


In [14]:
resultado.loc[resultado['P>|t|'] <= 0.1]

,coef,std err,t,P>|t|,[0.025,0.975]
indice,,,,,,
Intercept,7.32,0.04,165.58,0.00,7.24,7.41
"C(qtd_filhos, Treatment(0))[T.7]",-0.48,0.25,-1.93,0.05,-0.96,0.01
"C(qt_pessoas_residencia, Treatment(2))[T.9.0]",-0.48,0.25,-1.93,0.05,-0.96,0.01
"C(sexo, Treatment(0))[T.M]",0.48,0.01,33.56,0.00,0.45,0.51
"C(posse_de_imovel, Treatment(1))[T.False]",-0.09,0.01,-6.36,0.00,-0.11,-0.06
"C(tipo_renda, Treatment(0))[T.Empresário]",0.18,0.01,12.26,0.00,0.15,0.21
"C(tipo_renda, Treatment(0))[T.Pensionista]",0.42,0.24,1.79,0.07,-0.04,0.89
"C(tipo_renda, Treatment(0))[T.Servidor público]",0.09,0.02,4.33,0.00,0.05,0.13
"C(educacao, Treatment(2))[T.Pós graduação]",0.54,0.16,3.36,0.00,0.23,0.86


- Análisando o resultado acima, observa-se que os atributos: **data_ref, mau, posse_de_veiculo e estado_civil** podem ser desconsiderados para a análise, que só irão contribuir para a complexidade sem acrescentar muita informação nova. 
- Outro tratamento possível é agrupar os valores com estatística acima de 10 % para os atributos com classes importantes.



In [15]:
# Separando logo os atributos numéricos
numericos = 'idade + tempo_emprego + C(qtd_filhos, Treatment(0)) + C(qt_pessoas_residencia, Treatment(2))'
categoricos = 'C(sexo, Treatment(0)) + C(tipo_residencia, Treatment(1))'
categoricos += ' + C(posse_de_imovel, Treatment(1)) + C(tipo_renda, Treatment(0)) + C(educacao, Treatment(2))'
atributos = f'np.log(renda) ~ {numericos} + {categoricos}'
y, X = patsy.dmatrices(atributos, df)
# Treinando o modeo
model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.243
Model:                            OLS   Adj. R-squared:                  0.241
Method:                 Least Squares   F-statistic:                     137.3
Date:                Fri, 27 Oct 2023   Prob (F-statistic):               0.00
Time:                        16:44:13   Log-Likelihood:                -13198.
No. Observations:               12466   AIC:                         2.646e+04
Df Residuals:                   12436   BIC:                         2.668e+04
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
=====================================================================================================================
                                                        coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------
Intercept                                             7.3485      0.036    201.758      0.000       7.277       7.420
C(qtd_filhos, Treatment(0))[T.1]                     -0.0443      0.034     -1.310      0.190      -0.110       0.022
C(qtd_filhos, Treatment(0))[T.2]                      0.0838      0.099      0.843      0.399      -0.111       0.279
C(qtd_filhos, Treatment(0))[T.3]                     -0.4508      0.284     -1.585      0.113      -1.008       0.107
C(qtd_filhos, Treatment(0))[T.4]                     -0.1498      0.409     -0.366      0.714      -0.952       0.652
C(qtd_filhos, Treatment(0))[T.5]                     -0.0492      0.393     -0.125      0.900      -0.820       0.721
C(qtd_filhos, Treatment(0))[T.7]                     -0.4773      0.247     -1.932      0.053      -0.962       0.007
C(qtd_filhos, Treatment(0))[T.14]                     0.0462      0.175      0.264      0.792      -0.296       0.389
C(qt_pessoas_residencia, Treatment(2))[T.1.0]        -0.0026      0.018     -0.144      0.886      -0.038       0.033
C(qt_pessoas_residencia, Treatment(2))[T.3.0]         0.0445      0.035      1.260      0.208      -0.025       0.114
C(qt_pessoas_residencia, Treatment(2))[T.4.0]        -0.0275      0.101     -0.273      0.785      -0.225       0.170
C(qt_pessoas_residencia, Treatment(2))[T.5.0]         0.4685      0.290      1.618      0.106      -0.099       1.036
C(qt_pessoas_residencia, Treatment(2))[T.6.0]         0.3583      0.393      0.911      0.362      -0.412       1.129
C(qt_pessoas_residencia, Treatment(2))[T.7.0]        -0.5572      0.634     -0.878      0.380      -1.801       0.686
C(qt_pessoas_residencia, Treatment(2))[T.9.0]        -0.4773      0.247     -1.932      0.053      -0.962       0.007
C(qt_pessoas_residencia, Treatment(2))[T.15.0]        0.0462      0.175      0.264      0.792      -0.296       0.389
C(sexo, Treatment(0))[T.M]                            0.4831      0.013     35.950      0.000       0.457       0.509
C(tipo_residencia, Treatment(1))[T.Aluguel]          -0.0116      0.053     -0.217      0.828      -0.116       0.093
C(tipo_residencia, Treatment(1))[T.Com os pais]      -0.0731      0.029     -2.489      0.013      -0.131      -0.016
C(tipo_residencia, Treatment(1))[T.Comunitário]       0.0541      0.089      0.607      0.544      -0.121       0.229
C(tipo_residencia, Treatment(1))[T.Estúdio]          -0.1224      0.072     -1.704      0.088      -0.263       0.018
C(tipo_residencia, Treatment(1))[T.Governamental]    -0.0567      0.038     -1.502      0.133      -0.131       0.017
C(posse_de_imovel, Treatment(1))[T.False]            -0.0872      0.014     -6.434      0.000      -0.114      -0.061
C(tipo_renda, T

In [16]:
# Para a quantidade de filhos diferente de 7 .
df.loc[df['qt_pessoas_residencia'] != 7, 'qtd_filhos'] = 'outros'

# Para a quantidade de pessoas na residência diferente de 9 
df.loc[df['qt_pessoas_residencia'] != 9, 'qt_pessoas_residencia'] = 'outros'

# Para o tipo de renda
df.loc[(df['tipo_renda'] != 'Empresário') & (df['tipo_renda'] != 'Pensionista') & (df['tipo_renda'] != 'Servidor público'),'tipo_renda'] = 'outros'

# Para o nível de educação
df.loc[(df['educacao'] != 'Pós graduação') & (df['educacao'] != 'Superior completo'), 'educacao'] = 'outros'

# Para outro tipo de residência
df.loc[(df['tipo_residencia'] != 'Com os pais') & (df['tipo_residencia'] != 'Estúdio'), 'tipo_residencia'] = 'outros'

In [17]:
df.head()

,data_ref,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,mau,renda
0,2015-01-01,F,False,True,outros,outros,outros,Casado,outros,36,3.58,outros,False,3369.24
1,2015-01-01,M,True,True,outros,Empresário,outros,Casado,outros,42,0.86,outros,False,6096.14
2,2015-01-01,M,True,True,outros,outros,Superior completo,Casado,outros,31,8.07,outros,False,5658.98
3,2015-01-01,F,True,False,outros,Empresário,outros,Casado,outros,50,1.21,outros,False,7246.69
4,2015-01-01,M,False,False,outros,outros,outros,Casado,outros,52,13.87,outros,False,4017.37


In [18]:
# Separando logo os atributos numéricos
numericos = 'idade + tempo_emprego + C(qtd_filhos, Treatment(0)) + C(qt_pessoas_residencia, Treatment(0))'
categoricos = '+ C(sexo, Treatment(0)) + C(posse_de_imovel, Treatment(1)) + C(tipo_renda, Treatment(0)) '
categoricos += ' + C(educacao, Treatment(0)) + C(tipo_residencia, Treatment(0))'
atributos = f'np.log(renda) ~ {numericos} + {categoricos}'
y, X = patsy.dmatrices(atributos, df)
# Treinando o modeo
model = sm.OLS(y, X).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.241
Model:                            OLS   Adj. R-squared:                  0.240
Method:                 Least Squares   F-statistic:                     304.5
Date:                Fri, 27 Oct 2023   Prob (F-statistic):               0.00
Time:                        16:44:14   Log-Likelihood:                -13209.
No. Observations:               12466   AIC:                         2.645e+04
Df Residuals:                   12452   BIC:                         2.655e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                            6.4952      0.871      7.454      0.000       4.787       8.203
C(qtd_filhos, Treatment(0))[T.outros]                0.5621      0.699      0.804      0.421      -0.807       1.932
C(qt_pessoas_residencia, Treatment(0))[T.outros]     0.9534      0.494      1.929      0.054      -0.015       1.922
C(sexo, Treatment(0))[T.M]                           0.4864      0.013     36.500      0.000       0.460       0.513
C(posse_de_imovel, Treatment(1))[T.False]           -0.0901      0.013     -6.737      0.000      -0.116      -0.064
C(tipo_renda, Treatment(0))[T.Pensionista]           0.2057      0.233      0.882      0.378      -0.251       0.663
C(tipo_renda, Treatment(0))[T.Servidor público]     -0.0808      0.023     -3.496      0.000      -0.126      -0.036
C(tipo_renda, Treatment(0))[T.outros]               -0.1772      0.015    -12.158      0.000      -0.206      -0.149
C(educacao, Treatment(0))[T.Superior completo]      -0.4482      0.161     -2.789      0.005      -0.763      -0.133
C(educacao, Treatment(0))[T.outros]                 -0.5476      0.161     -3.410      0.001      -0.862      -0.233
C(tipo_residencia, Treatment(0))[T.Estúdio]         -0.0454      0.077     -0.592      0.554      -0.196       0.105
C(tipo_residencia, Treatment(0))[T.outros]           0.0734      0.029      2.511      0.012       0.016       0.131
idade                                                0.0046      0.001      6.374      0.000       0.003       0.006
tempo_emprego                                        0.0484      0.001     48.680      0.000       0.046       0.050
==============================================================================
Omnibus:                        1.065   Durbin-Watson:                   2.014
Prob(Omnibus):                  0.587   Jarque-Bera (JB):                1.033
Skew:                          -0.012   Prob(JB):                        0.597
Kurtosis:                       3.037   Cond. No.                     7.38e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.38e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

- Após as transformações e seleções dos dados, observa-se que o adj R2 foi muito pouco alterado, porém a quantidade de atributos diminuiu bastante, ou seja, o modelo ele ficou mais simples, porém apresentando um resultado praticamente igual. 
